In [1]:
from vplants.autowig import autowig

In [2]:
from path import path

rootdir = path('../../').abspath()

includedir = rootdir + "/core/build-scons/include"
headers = [str(f) for f in includedir.walkfiles('*.h')]
    
flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
         '-I/usr/include', '-I' + str(includedir), '-D__STDC_CONSTANT_MACROS',
         '-I/usr/local/lib/clang/3.7.0/include', '-D__STDC_LIMIT_MACROS']

force = False

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags, silent=True,
                  force_overload=True, bootstrap=3, cache=rootdir+'/.AutoWIG', force=force)

In [3]:
headers

['/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/mixture.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/event.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/predictor.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/distribution.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/sample_space.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/base.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/moment.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/slope_heuristic.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/data_frame.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/include/statiskit/core/estimator.h',
 '/home/pfernique/Desktop/Git/statiskit/core/build-scons/incl

In [4]:
asg.clean()

primaries = [header for header in asg.files(header=True) if header.is_primary]
for hdr in asg.files(header=True):
    if not hdr.is_primary and hdr.include in primaries:
        hdr.is_primary = True
        
for inc in asg.directories():
    if inc.as_include:
        if inc.globalname + 'c++/' in asg:
            for vsn in asg[inc.globalname + 'c++/'].directories:
                vsn.as_include = True
                for hdr in vsn.files:
                    if hdr.suffix == '':
                        hdr.is_primary = True
        elif inc.globalname + 'boost/' in asg:
            for hdr in sg[inc.globalname + 'boost/'].files:
                hdr.is_primary = True
                
for tpl in asg.classes('.*(unique|weak|shared)_ptr$', templated=True,
                       specialized=False):
    tpl.is_smart_pointer = True

In [5]:
autowig.back_end.plugin = 'boost_python:std_filter'
autowig.back_end(asg)

In [6]:
if 'struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >':
    asg['struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_row<double>, arma::op_strans> >':
    asg['struct ::arma::Base<double, arma::Op<arma::subview_row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Col<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Col<double>, arma::op_strans> >'].boost_python_export = False
if 'class ::arma::SpMat<double>' in asg:
    for mtd in asg['class ::arma::SpMat<double>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<double>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<double>' in asg:
    for mtd in asg['class ::arma::SpSubview<double>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False
if 'class ::arma::SpMat<unsigned long long>' in asg:
    for mtd in asg['class ::arma::SpMat<unsigned long long>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<unsigned long long>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<unsigned long long>' in asg:
    for mtd in asg['class ::arma::SpSubview<unsigned long long>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False

In [7]:
from vplants.autowig.boost_python_back_end import BoostPythonExportMappingFileProxy

autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/core/src/wrapper',
                 pattern = '.*statiskit.*',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:closure'
autowig.back_end(asg)
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/misc/src/wrapper',
                 proxy = BoostPythonExportMappingFileProxy,
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:module'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/wrapper/__core.cpp',
                 package = 'statiskit.core')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/wrapper/__misc.cpp',
                 package = 'statiskit.misc')
autowig.back_end.plugin = 'boost_python:import'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/statiskit/core/_core.py',
                 module = rootdir + '/core/src/wrapper/__core.cpp')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/statiskit/misc/_misc.py',
                 module = rootdir + '/misc/src/wrapper/__misc.cpp')

In [8]:
autowig.back_end.plugin = 'on_disk'
autowig.back_end(asg,
                 pattern = rootdir + '/(core|misc)/src/(wrapper|statiskit)/.*')

In [9]:
autowig.scons(rootdir + '/misc', '-j1')

In [10]:
autowig.scons(rootdir + '/core', '-j3')

In [11]:
asg['class ::std::exception']

class ::std::exception